In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: Path
    unzip_data_dir: Path
    all_schema: dict

In [19]:
from MLOps.constants import *
from MLOps.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(list_of_directories=[self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        data_validation_config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([data_validation_config.root_dir])
        return DataValidationConfig(
            root_dir=data_validation_config.root_dir,
            status_file=data_validation_config.STATUS_FILE,
            unzip_data_dir=data_validation_config.unzip_data_dir,
            all_schema=schema
        )

In [20]:
import os
from MLOps import logger

In [24]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = list(data.columns)
            
            schema = self.config.all_schema.keys()
            
            for cols in all_columns:
                if cols not in schema:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    break
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        
            return validation_status
        except Exception as e:
            raise e
        

In [26]:
try:
    logger.info(">>>>>>>>> stage Data Validation Stage started <<<<<<<<")
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    status = data_validation.validate_all_columns()
    logger.info(f"Data validation status: {status}")
    logger.info(">>>>>>>>> stage Data Validation Stage completed <<<<<<<<")
except Exception as e:
    logger.exception(e)
    raise e

2025-07-02 02:02:48,766 - MLOpsLogger - INFO - >>>>>>>>> stage Data Validation Stage started <<<<<<<<
Creating files
Directory already exists: artifacts
Creating files
Directory already exists: artifacts\data_validation
2025-07-02 02:02:48,832 - MLOpsLogger - INFO - Data validation status: True
2025-07-02 02:02:48,834 - MLOpsLogger - INFO - >>>>>>>>> stage Data Validation Stage completed <<<<<<<<
